In [49]:
# Import libraries
import pandas as pd
import re
import requests as req
from bs4 import BeautifulSoup as bs
import json

# Importar datos

In [50]:
# Import data
df=pd.read_csv("../data/Artworks.csv")

df

,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,...,ThumbnailURL,Circumference (cm),Depth (cm),Diameter (cm),Height (cm),Length (cm),Weight (kg),Width (cm),Seat Height (cm),Duration (sec.)
0,"Ferdinandsbrücke Project, Vienna, Austria, Ele...",Otto Wagner,6210,"(Austrian, 1841–1918)",(Austrian),(1841),(1918),(Male),1896,Ink and cut-and-pasted painted pages on paper,...,http://www.moma.org/media/W1siZiIsIjU5NDA1Il0s...,NaN,NaN,NaN,48.6000,NaN,NaN,168.9000,NaN,NaN
1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,7470,"(French, born 1944)",(French),(1944),(0),(Male),1987,Paint and colored pencil on print,...,http://www.moma.org/media/W1siZiIsIjk3Il0sWyJw...,NaN,NaN,NaN,40.6401,NaN,NaN,29.8451,NaN,NaN
2,"Villa near Vienna Project, Outside Vienna, Aus...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(Male),1903,"Graphite, pen, color pencil, ink, and gouache ...",...,http://www.moma.org/media/W1siZiIsIjk4Il0sWyJw...,NaN,NaN,NaN,34.3000,NaN,NaN,31.8000,NaN,NaN
3,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,"(French and Swiss, born Switzerland 1944)",(),(1944),(0),(Male),1980,Photographic reproduction with colored synthet...,...,http://www.moma.org/media/W1siZiIsIjEyNCJdLFsi...,NaN,NaN,NaN,50.8000,NaN,NaN,50.8000,NaN,NaN
4,"Villa, project, outside Vienna, Austria, Exter...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(Male),1903,"Graphite, color pencil, ink, and gouache on tr...",...,http://www.moma.org/media/W1siZiIsIjEyNiJdLFsi...,NaN,NaN,NaN,38.4000,NaN,NaN,19.1000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132398,An Episode in the Life of a Landscape Painter ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006,Book from a multiple of archival carrying case...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132399,Incense Sweaters & Ice (Installation),Martine Syms,68036,"(American, 1988)",(American),(1988),(0),(),2017,"Video (color, sound)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4500.0
132400,Just For You,Ken Friedman,2007,"(American, born 1949)",(American),(1939),(0),(Male),1967,NaN,...,NaN,NaN,0.0,NaN,0.0000,NaN,NaN,0.0000,NaN,NaN
132401,Marey and Demeny Chronophotographs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1892–1900,"35mm film (black and white, silent)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,420.0


# Evaluación de datos

In [51]:
artist = list(df.Artist.unique())

In [52]:
len(artist)

13863

In [53]:
df.shape

(132403, 29)

In [54]:
df.isnull().sum()

Title                     45
Artist                  1461
ConstituentID           1461
ArtistBio               4734
Nationality             1461
BeginDate               1461
EndDate                 1461
Gender                  1461
Date                    2166
Medium                 11225
Dimensions             11243
CreditLine              2924
AccessionNumber            0
Classification             0
Department                 0
DateAcquired            5994
Cataloged                  0
ObjectID                   0
URL                    57280
ThumbnailURL           68263
Circumference (cm)    132393
Depth (cm)            120479
Diameter (cm)         130985
Height (cm)            18765
Length (cm)           131665
Weight (kg)           132110
Width (cm)             19664
Seat Height (cm)      132403
Duration (sec.)       129258
dtype: int64

Trabajo con los artistas que han presentado trabajos en los últimos 50 años.

# CLEANING

Drop columns non useful

In [55]:
# DROP columns that are not useful for my analysis.
df1 = df.drop(["Title", "ConstituentID", "ArtistBio","Medium","ThumbnailURL", "BeginDate", "Gender", "EndDate", "Circumference (cm)", "Depth (cm)", "Diameter (cm)", "Height (cm)", "Length (cm)", "Weight (kg)", "Width (cm)", "Seat Height (cm)", "Duration (sec.)", "Dimensions", "CreditLine", "AccessionNumber", "Department", "DateAcquired", "Cataloged", "ObjectID", "URL"],axis=1)

In [56]:
df1.sample()

,Artist,Nationality,Date,Classification
87560,Susan Rothenberg,(American),1993,Illustrated Book


In [57]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132403 entries, 0 to 132402
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Artist          130942 non-null  object
 1   Nationality     130942 non-null  object
 2   Date            130237 non-null  object
 3   Classification  132403 non-null  object
dtypes: object(4)
memory usage: 4.0+ MB


Cleaning column date

In [58]:
# Cleaning "date" column to obtain an integer year.
date = list(df1.Date.unique())

In [59]:
date

['1896',
 '1987',
 '1903',
 '1980',
 '1976-77',
 '1968',
 '1900',
 '1978',
 '1905',
 '1906',
 '1979',
 '1980-81',
 '1918',
 '1970',
 '1975',
 '1984',
 '1986',
 '1974',
 'n.d.',
 'c. 1917',
 '1917',
 '1923',
 'Unknown',
 '1930',
 '1936',
 '1935',
 '1937',
 '1938',
 '1977',
 '1958',
 '1985',
 '1989',
 '1949',
 '1958–1964',
 'c. 1935',
 '1991',
 '1941',
 '1965',
 '1981',
 '1983',
 '1985–1988',
 'c. 1989-91',
 '1992',
 '1915-17',
 '1915–1917',
 'c. 1915-17',
 '1953',
 '1910',
 'c.1985',
 '1982–1986',
 '1982-86',
 '1945',
 '1985.',
 '1986.',
 '1923–1924',
 '.1-3 1987; .4 1990',
 '1990',
 '1976',
 '1995',
 '1927–1931',
 'c. 1929-30',
 '1964',
 '1959',
 'c. 1918-20',
 'c.1918-1920',
 '1939',
 'c.1976',
 '1975-79',
 '1993',
 '1989.',
 '1996',
 '1988',
 '1982-83',
 '1982–1983',
 '1952-53',
 '1921',
 '1957',
 '1972',
 '1956-57',
 '1924',
 '1962',
 '1925',
 '1960',
 '1969',
 '1963',
 '1994',
 ' 1961',
 '1960-61',
 '1952',
 'c. 1978-84',
 '1927',
 '1979–1985',
 'before 1933',
 '1961',
 '1929',
 'c

In [60]:
df1['year'] = df1['Date'].str.extract(r"(\d+)")

In [61]:
df1.head()

,Artist,Nationality,Date,Classification,year
0,Otto Wagner,(Austrian),1896,Architecture,1896
1,Christian de Portzamparc,(French),1987,Architecture,1987
2,Emil Hoppe,(Austrian),1903,Architecture,1903
3,Bernard Tschumi,(),1980,Architecture,1980
4,Emil Hoppe,(Austrian),1903,Architecture,1903


In [62]:
# Dropping column "Date"; I am going to use "year" column.
df1 = df1.drop(["Date"],axis=1)

In [63]:
df1.sample()

,Artist,Nationality,Classification,year
121036,"Pandit Pran Nath, Marian Zazeela",() (American),Audio,1971


In [64]:
# Drop rows that have NaN in year column.
df1 = df1[df1['year'].notna()]

In [65]:
df1['year'] = df1['year'].astype(int)

In [66]:
# I focus on the exhibitions from the last 50 years.
df2 = df1[df1["year"]>=1972]

In [67]:
df2.shape

(42319, 4)

In [68]:
# Elimino los NaN en la columna artist.
df2 = df2[df2['Artist'].notna()]

In [69]:
artist = list(df2.Artist)

In [70]:
artist

['Christian de Portzamparc',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard T

Cleaning column Artist.

In [71]:
# I first make a split on ",", and after that I make an "explode".
df2['artist'] = df2['Artist'].str.split(",")

In [72]:
art = list(df2.artist)

In [73]:
art

[['Christian de Portzamparc'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 

Cleaning column Country

In [74]:
#LIMPIEZA columna country.
df2['Country'] = df2['Nationality'].str.split(" ")

In [75]:
df3 = df2.explode('artist').explode('Country')

In [76]:
df3['artist'] = df3['artist'].str.strip()

In [77]:
art = list((df3.artist).unique())

In [78]:
#At the end I obtain this list.
art

['Christian de Portzamparc',
 'Bernard Tschumi',
 'Peter Eisenman',
 'Robert Cole',
 'Rem Koolhaas',
 'Madelon Vriesendorp',
 'Roger C. Ferri',
 'Michael Graves',
 'Aldo Rossi',
 'Gianni Braghieri',
 'M. Bosshard',
 'Steven Holl',
 'Paul Rudolph',
 'Peter Cook',
 'Diller + Scofidio',
 'Elizabeth Diller',
 'Ricardo Scofidio',
 'Zaha Hadid',
 'Fumihiko Maki',
 'Zoe Zenghelis',
 'Elia Zenghelis',
 'Mario Bellini',
 'Venturi and Rauch',
 'Robert Venturi',
 'John Rauch',
 'Denise Scott Brown',
 'Venturi',
 'Rauch and Scott Brown',
 'Emilio Ambasz',
 'Raimund Abraham',
 'Tadao Ando',
 'Arata Isozaki',
 'Neil M. Denari',
 'Mario Botta',
 'Alessandro Mendini',
 'Leon Krier',
 'Daniel Libeskind',
 'Thom Mayne',
 'Andrew Zago',
 'Morphosis',
 'Santa Monica',
 'CA',
 'Joel Sanders',
 'Massimo Scolari',
 'Richard Meier',
 'Rafael Viñoly',
 'Santiago Calatrava',
 'Norman Foster',
 'Ricardo Bofill',
 'Frank O. Gehry',
 'Wolf D. Prix',
 'Helmut Swiczinsky',
 'Cesar Pelli',
 'Chuck Hoberman',
 'Hans H

In [79]:
df3 = df3.drop(["Artist"],axis=1)

In [80]:
df3

,Nationality,Classification,year,artist,Country
1,(French),Architecture,1987,Christian de Portzamparc,(French)
3,(),Architecture,1980,Bernard Tschumi,()
5,(),Architecture,1976,Bernard Tschumi,()
6,(),Architecture,1976,Bernard Tschumi,()
7,(),Architecture,1976,Bernard Tschumi,()
...,...,...,...,...,...
132308,(Venezuelan),Print,2016,Christian Vinck Henriquez,(Venezuelan)
132309,(Venezuelan),Multiple,2016,Christian Vinck Henriquez,(Venezuelan)
132331,(American),Illustrated Book,2015,Laura Owens,(American)
132332,(American),Illustrated Book,2015,Laura Owens,(American)


In [81]:
list_countries = list(df3.Country)
list_countries

['(French)',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '(American)',
 '()',
 '(American)',
 '()',
 '(Dutch)',
 '(Dutch)',
 '(Dutch)',
 '(Dutch)',
 '(American)',
 '()',
 '(American)',
 '()',
 '(American)',
 '(American)',
 '(American)',
 '(American)',
 '(American)',
 '(Italian)',
 '(Italian)',
 '(Italian)',
 '(Italian)',
 '(Italian)',
 '(Italian)',
 '(Italian)',
 '(Italian)',
 '(Italian)',
 '(American)',
 '(American)',
 '(American)',
 '(American)',
 '(American)',
 '(American)',
 '(American)',
 '(American)',
 '(American)',
 '(American)',
 '(American)',
 '(American)',
 '(American)',
 '(American)',
 '(British)',
 '(American)',
 '(American)',
 '(American

In [82]:
#Change Nationality for country.
df3['country'] = df3['Country'].replace("(French)", "France").replace("(American)", "United States").replace("(Italian)", "Italy").replace("(Dutch)", "Netherlands").replace("(British)", "United Kingdom").replace("(Japanese)", "Japan").replace("(Argentine)", "Argentina").replace("(Swiss)", "Switzerland").replace("(Luxembourgish)", "Luxembourg").replace("(Spanish)", "Spain").replace("(Polish)", "Poland").replace("(Austrian)", "Austria").replace("(Iranian)", "Iran").replace("(German)", "Germany").replace("(Moroccan)", "Morocco").replace("(Danish)", "Denmark").replace("(Canadian)", "Canada").replace("(Brazilian)", "Brazil").replace("(Venezuelan)", "Venezuela").replace("(Belgian)", "Belgium").replace("(Norwegian)", "Norway").replace("(Finnish)", "Finland").replace("(Swedish)", "Sweden").replace("(Colombian)", "Colombia").replace("(Australian)", "Autralia").replace("(Yugoslav)", "Yugoslavia").replace("(Nationality", "None").replace("unknown)", "None").replace("(Hungarian)", "Hungary").replace("(Mexican)", "Mexico").replace("(Greek)", "Greece").replace("(Israeli)", "Israel").replace("(Croatian)", "Croatia").replace("(Cuban)", "Cuba").replace("(Thai)", "Thailand"). replace("(Czech)", "Czech Republic").replace("(Chilean)", "Chile").replace("(Various)", "None").replace("(Romanian)", "Romania").replace("(South", "South Africa").replace("African)", "South Africa").replace("(Russian)", "Russian Federation").replace("(Congolese)", "Central African Republic").replace("(Ukrainian)", "Ukraine").replace("(Peruvian)", "Peru").replace("(Indian)", "India").replace("(Cambodian)", "Cambodia").replace("(Haitian)", "Haiti").replace("(Scottish)", "United Kingdom").replace("(Korean)", "South Korea").replace("(Slovak)", "Slovak Republic").replace("(Estonian)", "Estonia").replace("(Pakistani)", "Pakistan").replace("(Icelandic)", "Iceland").replace("(Portuguese)", "Portugal").replace("(Chinese)", "China").replace("(Paraguayan)", "Paraguay").replace("(Uruguayan)", "Uruguay").replace("(Tunisian)", "Tunisia").replace("(Guyanese)", "Guatemala").replace("(Senegalese)", "Senegal").replace("(Bahamian)", "Cuba").replace("(Turkish)", "Turkey").replace("(Malian)", "Mali").replace("(Bulgarian)", "Bulgaria").replace("(New", "New Zealand").replace("Zealander)", "New Zealand").replace("(Irish)", "Ireland").replace("(Lebanese)", "Lebanon").replace("(English)", "United Kingdom"). replace("(Welsh)", "United Kingdom").replace("(Cypriot)", "Cyprus").replace("(Kenyan)", "Kenya").replace("(Syrian)", "Syrian Arab Republic").replace("(Saudi", "Saudi Arabia").replace("Arabian)", "Saudi Arabia").replace("(Slovenian)", "Slovenia").replace("(Nigerian)", "Central African Republic").replace("(Native", "United States").replace("American)", "United States").replace("(Bosnian)","Bosnia and Herzegovina").replace("Unknown)", "None").replace("(Kazakhstani)", "Kazakhstan").replace("(Kyrgyzstani)", "Kyrgyz Republic").replace("(Palestinian)", "Israel").replace("(Catalan)", "Spain").replace("(Vietnamese)", "Vietnam").replace("(Indonesian)", "Indonesia").replace("(Filipino)", "Philippines").replace("(Guatemalan)", "Guatemala").replace("(Egyptian)", "Egypt").replace("(Ghanaian)", "Ghana").replace("(Serbian)", "Bosnia and Herzegovina").replace("(Mauritanian)", "Mauritania").replace("(Ugandan)", "Uganda").replace("(Taiwanese)", "Taiwan").replace("(Albanian)", "Albania").replace("(Malaysian)", "Malaysia").replace("(Iraqi)", "Iraq").replace("()", "None").replace("(Tajik)", "Tajikistan").replace("(Singaporean)", "Singapore").replace("(Puerto", "Dominican Republic").replace("Rican)","Dominican Republic").replace("(Zimbabwean)", "Zimbabwe").replace("(Cameroonian)", "Cameroon").replace("(Rwandan)", "Rwanda").replace("(Mozambican)", "Mozambique").replace("(Macedonian)", "Greece").replace("(Georgian)", "None").replace("(Sudanese)", "Sudan")

In [83]:
list_country = list((df3['country']).unique())

In [84]:
list_country

['France',
 'None',
 'United States',
 'Netherlands',
 'Italy',
 'United Kingdom',
 'Japan',
 'Argentina',
 'Switzerland',
 'Luxembourg',
 'Spain',
 'Austria',
 'Poland',
 'Iran',
 'Germany',
 'Canada',
 'Belgium',
 'Norway',
 'Brazil',
 'Finland',
 'Denmark',
 'Sweden',
 'Morocco',
 'Colombia',
 'Autralia',
 'Yugoslavia',
 'Hungary',
 'Mexico',
 'Greece',
 'Israel',
 'Croatia',
 'Cuba',
 'Chile',
 'Russian Federation',
 'Thailand',
 'Czech Republic',
 'Romania',
 'South Africa',
 'Central African Republic',
 'Ukraine',
 'Peru',
 'India',
 'Cambodia',
 'Venezuela',
 'Haiti',
 'South Korea',
 'Slovak Republic',
 'Estonia',
 'Pakistan',
 'Iceland',
 'Portugal',
 'China',
 'Paraguay',
 'Uruguay',
 'Tunisia',
 'Guatemala',
 'Senegal',
 'Turkey',
 'Mali',
 'Bulgaria',
 'New Zealand',
 'Ireland',
 'Lebanon',
 'Cyprus',
 'Kenya',
 'Taiwan',
 'Slovenia',
 'Egypt',
 'Albania',
 'Tajikistan',
 '(Algerian)',
 '(Lithuanian)',
 '(Namibian)',
 'Bosnia and Herzegovina',
 'Ghana',
 '(Afghan)',
 'Kyrgy

In [85]:
df3 = df3.drop(["Country"],axis=1)

In [86]:
df3 = df3.drop(["Nationality"],axis=1)

In [87]:
df3

,Classification,year,artist,country
1,Architecture,1987,Christian de Portzamparc,France
3,Architecture,1980,Bernard Tschumi,None
5,Architecture,1976,Bernard Tschumi,None
6,Architecture,1976,Bernard Tschumi,None
7,Architecture,1976,Bernard Tschumi,None
...,...,...,...,...
132308,Print,2016,Christian Vinck Henriquez,Venezuela
132309,Multiple,2016,Christian Vinck Henriquez,Venezuela
132331,Illustrated Book,2015,Laura Owens,United States
132332,Illustrated Book,2015,Laura Owens,United States


In [92]:
df3.drop(df3.loc[df3['country']=='None'].index, inplace=True)

In [97]:
df4= df3.drop_duplicates()

In [98]:
df4

,Classification,year,artist,country
1,Architecture,1987,Christian de Portzamparc,France
66,Architecture,1987,Rem Koolhaas,Netherlands
66,Architecture,1987,Madelon Vriesendorp,Netherlands
67,Architecture,1979,Roger C. Ferri,United States
72,Architecture,1978,Michael Graves,United States
...,...,...,...,...
132284,Multiple,2016,Nicolás Paris,Colombia
132288,Illustrated Book,2016,Nicolás Paris,Colombia
132289,Print,2016,Rosângela Rennó,Brazil
132290,Print,2016,Christian Vinck Henriquez,Venezuela


In [99]:
df4.reset_index()

,index,Classification,year,artist,country
0,1,Architecture,1987,Christian de Portzamparc,France
1,66,Architecture,1987,Rem Koolhaas,Netherlands
2,66,Architecture,1987,Madelon Vriesendorp,Netherlands
3,67,Architecture,1979,Roger C. Ferri,United States
4,72,Architecture,1978,Michael Graves,United States
...,...,...,...,...,...
17237,132284,Multiple,2016,Nicolás Paris,Colombia
17238,132288,Illustrated Book,2016,Nicolás Paris,Colombia
17239,132289,Print,2016,Rosângela Rennó,Brazil
17240,132290,Print,2016,Christian Vinck Henriquez,Venezuela


In [ ]:
df5 = df4.groupby

# Enrich data base with scraping

EDUCATIONAL DATA (Creativity index)

In [100]:
url = 'http://chartsbin.com/view/41109'

In [101]:
html=req.get(url).content

In [102]:
soup=bs(html, 'html.parser')
type(soup)

bs4.BeautifulSoup

In [103]:
tabla=soup.find('table')

type(tabla)

bs4.element.Tag

In [104]:
filas=tabla.find_all('tr')

elementos=[[e.text for e in fila] for fila in filas]

elementos

[['Rank',
  'Country',
  'Technology',
  'Talent',
  'Tolerance',
  'Global Creativity Index'],
 ['1', 'Australia', '7', '1', '4', '0.97'],
 ['2', 'United States', '4', '3', '11', '0.95'],
 ['3', 'New Zealand', '7', '8', '3', '0.949'],
 ['4', 'Canada', '13', '14', '1', '0.92'],
 ['5', 'Denmark', '10', '6', '13', '0.917'],
 ['5', 'Finland', '5', '3', '20', '0.917'],
 ['7', 'Sweden', '11', '8', '10', '0.915'],
 ['8', 'Iceland', '26', '2', '2', '0.913'],
 ['9', 'Singapore', '7', '5', '23', '0.896'],
 ['10', 'Netherlands', '20', '11', '6', '0.889'],
 ['11', 'Norway', '18', '12', '9', '0.883'],
 ['12', 'United Kingdom', '15', '20', '5', '0.881'],
 ['13', 'Ireland', '23', '21', '7', '0.845'],
 ['14', 'Germany', '7', '28', '18', '0.837'],
 ['16', 'Switzerland', '19', '22', '17', '0.822'],
 ['16', 'France', '16', '26', '16', '0.822'],
 ['16', 'Slovenia', '17', '8', '35', '0.822'],
 ['18', 'Belgium', '28', '18', '14', '0.817'],
 ['19', 'Spain', '31', '19', '12', '0.811'],
 ['20', 'Austria', '12

In [105]:
col_names=elementos[0]

data=elementos[1:]

df=pd.DataFrame(data, columns=col_names)

df.head(10)


,Rank,Country,Technology,Talent,Tolerance,Global Creativity Index
0,1,Australia,7,1,4,0.97
1,2,United States,4,3,11,0.95
2,3,New Zealand,7,8,3,0.949
3,4,Canada,13,14,1,0.92
4,5,Denmark,10,6,13,0.917
5,5,Finland,5,3,20,0.917
6,7,Sweden,11,8,10,0.915
7,8,Iceland,26,2,2,0.913
8,9,Singapore,7,5,23,0.896
9,10,Netherlands,20,11,6,0.889


ECONOMY

I find the GDP of 4 years distributed on the last 50 years to make the avareage and obtain a value closer to the reality of these years. I have chosen these years; 1980, 1999, 2010 and 2020.

Año 1980

In [106]:
url = 'https://countryeconomy.com/gdp?year=1980'

In [107]:
html=req.get(url).content

In [108]:
soup=bs(html, 'html.parser')
type(soup)

bs4.BeautifulSoup

In [109]:
tabla=soup.find('table')

type(tabla)

bs4.element.Tag

In [110]:
filas=tabla.find_all('tr')

elementos=[[e.text.replace("\xa0M€", "").replace("\xa0M$", "").replace("[+]", "").strip() for e in fila] for fila in filas]

elementos

[['Countries', 'Date', 'Annual GDP', 'Annual GDP', 'GDP Growth (%)'],
 ['United States', '1980', '', '', '$2,857,300M', '', '-0.3%', ''],
 ['Euro zone', '1980', '', '', '$2,962,181M', '', '', ''],
 ['United Kingdom', '1980', '€406,184M', '', '$603,983M', '', '-2.0%', ''],
 ['Germany', '1980', '€610,968M', '', '$853,705M', '', '1.4%', ''],
 ['France', '1980', '€504,931M', '', '$701,305M', '', '1.6%', ''],
 ['Japan', '1980', '€770,809M', '', '$1,127,880M', '', '2.8%', ''],
 ['Spain', '1980', '€159,100M', '', '$230,759M', '', '1.2%', ''],
 ['Italy', '1980', '€331,149M', '', '$482,019M', '', '19.6%', ''],
 ['Portugal', '1980', '€8,236M', '', '$32,607M', '', '4.6%', ''],
 ['Greece', '1980', '€7,110M', '', '$56,529M', '', '0.7%', ''],
 ['Ireland', '1980', '€13,352M', '', '$21,425M', '', '3.1%', ''],
 ['Andorra', '1980', '€238M', '', '$446M', '', '2.2%', ''],
 ['United Arab Emirates', '1980', '', '', '$40,415M', '', '-1.8%', ''],
 ['Afghanistan', '1980', '', '', '$3,642M', '', '-3.7%', ''],
 

In [111]:
gdp_80=[]


for e in elementos:
    tmp=[]

    for st in e:
        if st!='':
            tmp.append(st)
    gdp_80.append(tmp)
    
gdp_80[:5]

[['Countries', 'Date', 'Annual GDP', 'Annual GDP', 'GDP Growth (%)'],
 ['United States', '1980', '$2,857,300M', '-0.3%'],
 ['Euro zone', '1980', '$2,962,181M'],
 ['United Kingdom', '1980', '€406,184M', '$603,983M', '-2.0%'],
 ['Germany', '1980', '€610,968M', '$853,705M', '1.4%']]

In [114]:
col_names=gdp_80[0]

data=gdp_80[1:]

df_80=pd.DataFrame(data, columns=col_names)

df_80.head(10)

,Countries,Date,Annual GDP,Annual GDP,GDP Growth (%)
0,United States,1980,"$2,857,300M",-0.3%,None
1,Euro zone,1980,"$2,962,181M",None,None
2,United Kingdom,1980,"€406,184M","$603,983M",-2.0%
3,Germany,1980,"€610,968M","$853,705M",1.4%
4,France,1980,"€504,931M","$701,305M",1.6%
5,Japan,1980,"€770,809M","$1,127,880M",2.8%
6,Spain,1980,"€159,100M","$230,759M",1.2%
7,Italy,1980,"€331,149M","$482,019M",19.6%
8,Portugal,1980,"€8,236M","$32,607M",4.6%
9,Greece,1980,"€7,110M","$56,529M",0.7%


Año 1999

In [115]:
url = 'https://countryeconomy.com/gdp?year=1999'

In [116]:
html=req.get(url).content

In [117]:
soup=bs(html, 'html.parser')
type(soup)

bs4.BeautifulSoup

In [118]:
tabla=soup.find('table')

type(tabla)

bs4.element.Tag

In [119]:
filas=tabla.find_all('tr')

elementos=[[e.text.replace("\xa0M€", "").replace("\xa0M$", "").replace("[+]", "").strip() for e in fila] for fila in filas]

elementos

[['Countries', 'Date', 'Annual GDP', 'Annual GDP', 'GDP Growth (%)'],
 ['United States', '1999', '€9,036,592M', '', '$9,631,200M', '', '4.8%', ''],
 ['Euro zone', '1999', '€6,456,826M', '', '$7,113,797M', '', '2.9%', ''],
 ['United Kingdom', '1999', '€1,578,395M', '', '$1,682,590M', '', '3.3%', ''],
 ['Germany', '1999', '€2,059,480M', '', '$2,197,130M', '', '1.9%', ''],
 ['France', '1999', '€1,400,999M', '', '$1,492,670M', '', '3.4%', ''],
 ['Japan', '1999', '€4,161,747M', '', '$4,635,980M', '', '-0.2%', ''],
 ['Spain', '1999', '€595,723M', '', '$635,968M', '', '4.5%', ''],
 ['Italy', '1999', '€1,175,150M', '', '$1,253,690M', '', '1.6%', ''],
 ['Portugal', '1999', '€119,603M', '', '$127,597M', '', '3.9%', ''],
 ['Greece', '1999', '€139,945M', '', '$148,151M', '', '3.1%', ''],
 ['Ireland', '1999', '€92,791M', '', '$98,993M', '', '10.5%', ''],
 ['Andorra', '1999', '€1,473M', '', '$1,240M', '', '4.1%', ''],
 ['United Arab Emirates', '1999', '€77,749M', '', '$82,865M', '', '3.8%', ''],
 ['

In [120]:
gdp_99=[]


for e in elementos:
    tmp=[]

    for st in e:
        if st!='':
            tmp.append(st)
    gdp_99.append(tmp)
    
gdp_99[:5]

[['Countries', 'Date', 'Annual GDP', 'Annual GDP', 'GDP Growth (%)'],
 ['United States', '1999', '€9,036,592M', '$9,631,200M', '4.8%'],
 ['Euro zone', '1999', '€6,456,826M', '$7,113,797M', '2.9%'],
 ['United Kingdom', '1999', '€1,578,395M', '$1,682,590M', '3.3%'],
 ['Germany', '1999', '€2,059,480M', '$2,197,130M', '1.9%']]

In [121]:
col_names=gdp[0]

data=gdp_99[1:]

df_99=pd.DataFrame(data, columns=col_names)

df_99.head(10)

NameError: name 'gdp' is not defined

Año 2010

In [122]:
url = 'https://countryeconomy.com/gdp?year=2010'

In [123]:
html=req.get(url).content

In [124]:
soup=bs(html, 'html.parser')
type(soup)

bs4.BeautifulSoup

In [125]:
tabla=soup.find('table')

type(tabla)

bs4.element.Tag

In [126]:
filas=tabla.find_all('tr')

elementos=[[e.text.replace("\xa0M€", "").replace("\xa0M$", "").replace("[+]", "").strip() for e in fila] for fila in filas]

elementos

[['Countries', 'Date', 'Annual GDP', 'Annual GDP', 'GDP Growth (%)'],
 ['United States', '2010', '€11,351,739M', '', '$15,049,000M', '', '2.7%', ''],
 ['Euro zone', '2010', '€9,472,472M', '', '$12,629,070M', '', '2.2%', ''],
 ['United Kingdom', '2010', '€1,872,176M', '', '$2,484,020M', '', '2.1%', ''],
 ['Germany', '2010', '€2,564,400M', '', '$3,402,440M', '', '4.2%', ''],
 ['France', '2010', '€1,995,289M', '', '$2,642,440M', '', '1.9%', ''],
 ['Japan', '2010', '€4,149,900M', '', '$5,759,070M', '', '4.7%', ''],
 ['Spain', '2010', '€1,072,709M', '', '$1,423,270M', '', '0.2%', ''],
 ['Italy', '2010', '€1,611,279M', '', '$2,137,850M', '', '1.7%', ''],
 ['Portugal', '2010', '€179,611M', '', '$238,308M', '', '1.7%', ''],
 ['Greece', '2010', '€224,124M', '', '$297,368M', '', '-5.5%', ''],
 ['Ireland', '2010', '€167,363M', '', '$222,057M', '', '1.8%', ''],
 ['Andorra', '2010', '€2,534M', '', '$3,446M', '', '-2.0%', ''],
 ['United Arab Emirates',
  '2010',
  '€218,592M',
  '',
  '$289,787M',
 

In [127]:
gdp_10=[]


for e in elementos:
    tmp=[]

    for st in e:
        if st!='':
            tmp.append(st)
    gdp_10.append(tmp)
    
gdp_10[:5]

[['Countries', 'Date', 'Annual GDP', 'Annual GDP', 'GDP Growth (%)'],
 ['United States', '2010', '€11,351,739M', '$15,049,000M', '2.7%'],
 ['Euro zone', '2010', '€9,472,472M', '$12,629,070M', '2.2%'],
 ['United Kingdom', '2010', '€1,872,176M', '$2,484,020M', '2.1%'],
 ['Germany', '2010', '€2,564,400M', '$3,402,440M', '4.2%']]

In [128]:
col_names=gdp_10[0]

data=gdp_10[1:]

df_10=pd.DataFrame(data, columns=col_names)

df_10.head(10)

,Countries,Date,Annual GDP,Annual GDP,GDP Growth (%)
0,United States,2010,"€11,351,739M","$15,049,000M",2.7%
1,Euro zone,2010,"€9,472,472M","$12,629,070M",2.2%
2,United Kingdom,2010,"€1,872,176M","$2,484,020M",2.1%
3,Germany,2010,"€2,564,400M","$3,402,440M",4.2%
4,France,2010,"€1,995,289M","$2,642,440M",1.9%
5,Japan,2010,"€4,149,900M","$5,759,070M",4.7%
6,Spain,2010,"€1,072,709M","$1,423,270M",0.2%
7,Italy,2010,"€1,611,279M","$2,137,850M",1.7%
8,Portugal,2010,"€179,611M","$238,308M",1.7%
9,Greece,2010,"€224,124M","$297,368M",-5.5%


Año 2020

In [129]:
url='https://countryeconomy.com/gdp?year=2020'

In [130]:
html=req.get(url).content

In [131]:
soup=bs(html, 'html.parser')
type(soup)

bs4.BeautifulSoup

In [132]:
tabla=soup.find('table')

type(tabla)

bs4.element.Tag

In [133]:
filas=tabla.find_all('tr')

elementos=[[e.text.replace("\xa0M€", "").replace("\xa0M$", "").replace("[+]", "").strip() for e in fila] for fila in filas]

elementos

[['Countries', 'Date', 'Annual GDP', 'Annual GDP', 'GDP Growth (%)'],
 ['United States',
  '2020',
  '€18,292,506M',
  '',
  '$20,893,700M',
  '',
  '-3.4%',
  ''],
 ['Euro zone', '2020', '€11,400,271M', '', '$13,021,206M', '', '-6.4%', ''],
 ['United Kingdom', '2020', '€2,423,371M', '', '$2,709,680M', '', '-9.7%', ''],
 ['Germany', '2020', '€3,367,560M', '', '$3,843,340M', '', '-4.6%', ''],
 ['France', '2020', '€2,302,860M', '', '$2,624,420M', '', '-7.9%', ''],
 ['Japan', '2020', '€4,432,007M', '', '$5,045,100M', '', '-4.6%', ''],
 ['Spain', '2020', '€1,121,948M', '', '$1,280,460M', '', '-10.8%', ''],
 ['Italy', '2020', '€1,653,577M', '', '$1,884,940M', '', '-8.9%', ''],
 ['Portugal', '2020', '€200,088M', '', '$228,356M', '', '-8.4%', ''],
 ['Greece', '2020', '€165,326M', '', '$189,259M', '', '-9.0%', ''],
 ['Ireland', '2020', '€372,869M', '', '$425,549M', '', '5.9%', ''],
 ['Andorra', '2020', '€2,502M', '', '$2,858M', '', '-12.0%', ''],
 ['United Arab Emirates',
  '2020',
  '€314,191

In [134]:
gdp_20=[]


for e in elementos:
    tmp=[]

    for st in e:
        if st!='':
            tmp.append(st)
    gdp_20.append(tmp)
    
gdp_20[:5]

[['Countries', 'Date', 'Annual GDP', 'Annual GDP', 'GDP Growth (%)'],
 ['United States', '2020', '€18,292,506M', '$20,893,700M', '-3.4%'],
 ['Euro zone', '2020', '€11,400,271M', '$13,021,206M', '-6.4%'],
 ['United Kingdom', '2020', '€2,423,371M', '$2,709,680M', '-9.7%'],
 ['Germany', '2020', '€3,367,560M', '$3,843,340M', '-4.6%']]

In [135]:
col_names=gdp_20[0]

data=gdp_20[1:]

df_20=pd.DataFrame(data, columns=col_names)

df_20.head(10)

,Countries,Date,Annual GDP,Annual GDP,GDP Growth (%)
0,United States,2020,"€18,292,506M","$20,893,700M",-3.4%
1,Euro zone,2020,"€11,400,271M","$13,021,206M",-6.4%
2,United Kingdom,2020,"€2,423,371M","$2,709,680M",-9.7%
3,Germany,2020,"€3,367,560M","$3,843,340M",-4.6%
4,France,2020,"€2,302,860M","$2,624,420M",-7.9%
5,Japan,2020,"€4,432,007M","$5,045,100M",-4.6%
6,Spain,2020,"€1,121,948M","$1,280,460M",-10.8%
7,Italy,2020,"€1,653,577M","$1,884,940M",-8.9%
8,Portugal,2020,"€200,088M","$228,356M",-8.4%
9,Greece,2020,"€165,326M","$189,259M",-9.0%
